In [4]:
%pip install gradio

  Using cached gradio-5.8.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached anyio-4.7.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.5.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached huggingface_hub-0.26.5-py3-none-any.whl.metadata (13 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
  Using cached orjson-3.10.12-cp312-none-win_amd64.whl.metadata (42 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.19-py3-none-any.whl.metadat

In [1]:
import pickle
import gradio
import cv2
import numpy as np

c:\Users\yoshu\OneDrive\Documents\GitHub\CV-Final-Project-main\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('model.pickle', 'rb') as file:
    model = pickle.load(file)
with open('label_encoder.pkl', 'rb') as le:
    label = pickle.load(le)

In [3]:
model

RandomForestClassifier(random_state=42)

In [4]:
label

LabelEncoder()

In [5]:
def extract_features(image):
    image = np.array(image)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (3, 3), 0)
    sift = cv2.SIFT_create()
    orb = cv2.ORB_create()
    sobelx = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
    sobel_edges = cv2.magnitude(sobelx, sobely).astype(np.uint8)

    sift_keypoints, sift_descriptors = sift.detectAndCompute(sobel_edges, None)
    orb_keypoints, orb_descriptors = orb.detectAndCompute(sobel_edges, None)
    if sift_descriptors is not None and orb_descriptors is not None:
        sift_mean = np.mean(sift_descriptors, axis=0)
        orb_mean = np.mean(orb_descriptors, axis=0)
        combined_features = np.hstack((sift_mean, orb_mean))
    else:
        print("failed")
    combined_features = combined_features.reshape(1, -1)
    classify = model.predict(combined_features)
    class_label = label.inverse_transform(classify)[0]
    return (f'Predicted Category: {class_label}')
    

In [6]:
interface = gradio.Interface(
    fn=extract_features,
    inputs=gradio.Image(type="pil"),
    outputs="text",
    title="Artwork Classification",
    description="Upload an image of an artwork to classify it into categories like drawings, painting, etc.",
)

In [9]:
interface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.
